In [1]:
import cmdstanpy
import numpy as np
import pandas as pd

from IPython.display import clear_output
from itertools import product
cmdstanpy.install_cmdstan();

CmdStan install directory: /Users/igor.michels/.cmdstan
CmdStan version 2.34.1 already installed
Test model compilation


In [2]:
NUM_SEASONS = 1
NUM_TEAMS = 20

In [3]:
def generate_matchups(n_equipes, habilidades, home_force):
    if home_force is None: home_force = 0
    confrontos = []
    for equipe1, equipe2 in product(range(1, n_equipes + 1), repeat=2):
        if equipe1 == equipe2: continue
        home_score = np.random.poisson(lam=(habilidades[equipe1 - 1] + home_force) / habilidades[equipe2 - 1])
        away_score = np.random.poisson(lam=habilidades[equipe2 - 1] / habilidades[equipe1 - 1])
        confrontos.append({
            'equipe1': equipe1,
            'equipe2': equipe2,
            'gols_equipe1': home_score,
            'gols_equipe2': away_score
        })

    df_confrontos = pd.DataFrame(confrontos)

    return df_confrontos

def generate_seasons(n_equipes, habilidades, n_seasons, home_force=None):
    df = pd.DataFrame()
    for _ in range(n_seasons):
        df = pd.concat([df, generate_matchups(n_equipes, habilidades, home_force)])
    
    return df

In [4]:
habilidades = abs(np.random.normal(0, 1, NUM_TEAMS))
habilidades /= habilidades[0]
df = generate_seasons(NUM_TEAMS, habilidades, NUM_SEASONS)

modelo_compilado = cmdstanpy.CmdStanModel(stan_file='../models/Poisson_model.stan', force_compile=False)
dados_modelo = {
    'num_jogos': len(df),
    'num_equipes': NUM_TEAMS,
    'equipe1': df['equipe1'].values,
    'equipe2': df['equipe2'].values,
    'gols_equipe1': df['gols_equipe1'].values,
    'gols_equipe2': df['gols_equipe2'].values,
}

ajuste = modelo_compilado.sample(data=dados_modelo, chains=4, iter_warmup=2000, iter_sampling=10000, show_console=False)
results = ajuste.summary() / ajuste.summary()['Mean']['habilidade[1]']
inside_ci = (habilidades > results['5%'].values[1:])
inside_ci *= ((habilidades < results['95%'].values[1:]))

clear_output()
sum(inside_ci) / NUM_TEAMS

1.0

In [5]:
home_force = abs(np.random.normal(0, 1))
habilidades = abs(np.random.normal(0, 1, NUM_TEAMS))

habilidades /= home_force
home_force = 1

df = generate_seasons(NUM_TEAMS, habilidades, NUM_SEASONS, home_force)
modelo_compilado = cmdstanpy.CmdStanModel(stan_file='../models/Poisson_model_2.stan', force_compile=False)
dados_modelo = {
    'num_jogos': len(df),
    'num_equipes': NUM_TEAMS,
    'equipe1': df['equipe1'].values,
    'equipe2': df['equipe2'].values,
    'gols_equipe1': df['gols_equipe1'].values,
    'gols_equipe2': df['gols_equipe2'].values,
}

ajuste = modelo_compilado.sample(data=dados_modelo, chains=4, iter_warmup=2000, iter_sampling=10000, show_console=False)
inside_ci = (habilidades > ajuste.summary()['5%'].values[1:])
inside_ci *= ((habilidades < ajuste.summary()['95%'].values[1:]))

clear_output()
sum(inside_ci) / NUM_TEAMS

0.95